In [5]:
# =================== HINDI VIDEO DUBBING PIPELINE ===================


import os, shutil, time
import whisper
from moviepy import VideoFileClip, AudioFileClip, concatenate_audioclips
from transformers import MarianMTModel, MarianTokenizer
from gtts import gTTS
from tqdm import tqdm

# 🎛️ Configuration
VIDEO_PATH = "input_video.mp4"
TEMP_AUDIO = "temp_audio.wav"
TEMP_CLIPS_DIR = "temp_audio_clips"
OUTPUT_VIDEO = "dubbed_video.mp4"
MODEL_SIZE = "base"
TRANSLATION_MODEL_NAME = 'Helsinki-NLP/opus-mt-en-hi'

try:
    if not os.path.exists(VIDEO_PATH):
        raise FileNotFoundError(f"Input video '{VIDEO_PATH}' not found.")

    # 🔊 Step 1: Extract audio
    print("🔊 Extracting audio...")
    video_clip = VideoFileClip(VIDEO_PATH)
    video_clip.audio.write_audiofile(TEMP_AUDIO, codec='pcm_s16le', logger=None)
    video_clip.close()

    # 🎙️ Step 2: Transcribe
    print(f"🎙️ Transcribing with Whisper '{MODEL_SIZE}'...")
    model = whisper.load_model(MODEL_SIZE)
    segments = model.transcribe(TEMP_AUDIO, word_timestamps=True)['segments']

    # 🌐 Step 3: Translate
    print("🌐 Translating to Hindi...")
    tokenizer = MarianTokenizer.from_pretrained(TRANSLATION_MODEL_NAME)
    translator = MarianMTModel.from_pretrained(TRANSLATION_MODEL_NAME)
    for segment in tqdm(segments, desc="Translating"):
        text = segment['text'].strip()
        if text:
            inputs = tokenizer(text, return_tensors="pt", padding=True)
            translated = translator.generate(**inputs)
            segment['translated_text'] = tokenizer.decode(translated[0], skip_special_tokens=True)

    # 🎤 Step 4: Generate Hindi audio
    print("🎤 Generating Hindi audio clips...")
    os.makedirs(TEMP_CLIPS_DIR, exist_ok=True)
    clip_paths = []
    for i, segment in enumerate(tqdm(segments, desc="Generating Audio")):
        text = segment.get('translated_text', '').strip()
        if text:
            path = os.path.join(TEMP_CLIPS_DIR, f"clip_{i}.mp3")
            try:
                gTTS(text, lang='hi').save(path)
                clip_paths.append(path)
            except Exception as e:
                print(f"❌ Segment {i} failed: {e}")

    # 🎬 Step 5: Create final video
    print("🎬 Creating dubbed video...")
    audio_clips = []
    for path in clip_paths:
        try:
            clip = AudioFileClip(path)
            audio_clips.append(clip)
        except Exception as e:
            print(f"⚠️ Error loading clip {path}: {e}")

    final_audio = concatenate_audioclips(audio_clips)
    original_clip = VideoFileClip(VIDEO_PATH)
    dubbed_clip = original_clip.set_audio(final_audio)
    dubbed_clip.write_videofile(OUTPUT_VIDEO, codec="libx264", audio_codec="aac", logger=None)

    # Close all clips
    original_clip.close()
    dubbed_clip.close()
    for clip in audio_clips:
        clip.close()

except Exception as e:
    print(f"❌ Error: {e}")

finally:
    print("🧹 Cleaning up...")
    time.sleep(1)  # Give system time to release file handles
    for path in [TEMP_AUDIO, TEMP_CLIPS_DIR]:
        try:
            if os.path.isdir(path):
                shutil.rmtree(path)
            elif os.path.isfile(path):
                os.remove(path)
        except Exception as cleanup_err:
            print(f"⚠️ Cleanup issue: {cleanup_err}")
    print("✅ Done.")



🔊 Extracting audio...
🎙️ Transcribing with Whisper 'base'...


C:\Users\Gaurav\AppData\Roaming\Python\Python310\site-packages\whisper\transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


🌐 Translating to Hindi...


Translating: 100%|██████████| 3/3 [00:04<00:00,  1.66s/it]


🎤 Generating Hindi audio clips...


Generating Audio: 100%|██████████| 3/3 [00:06<00:00,  2.04s/it]


🎬 Creating dubbed video...
❌ Error: 'VideoFileClip' object has no attribute 'set_audio'
🧹 Cleaning up...
⚠️ Cleanup issue: [WinError 32] The process cannot access the file because it is being used by another process: 'temp_audio_clips\\clip_0.mp3'
✅ Done.
